#Disulfide Docking.
###This notebook contains the script which docks a receptor ontu a larger disulfide ligand such as GSSG or Cystine. The example used here is the receptor 3NT1 docked onto the cystine ligand. 

###The centers are specified and could be changed so that the receptor is only run through those particular places to find where the best energy is. In order to get better results, the exhaustiveness and the number of binding configurations can be increased. However, for faster results it is better to use smaller numbers.

#### The script in this notebook should work on any computer so far as the rootdir is changed to match the location of the data on the particular computer being used.

In [2]:
use Modern::Perl;
use HackaMol;
use HackaMol::X::Vina;
use Math::Vector::Real;

In [5]:
my $rootdir = '/Users/chem_student/Fadilahs_Space/Disulfide_Docking/data/';

my $receptor = $rootdir . "receptors/pdbqts/3NT1.pdbqt";
my $ligand   = $rootdir . "ligands/pdbqts/cystine_90.pdbqt",

my $rmol     = HackaMol -> new( hush_read=>1 ) -> read_file_mol( $receptor );
my $lmol     = HackaMol -> new( hush_read=>1 ) -> read_file_mol( $ligand );
my $fh = $lmol->print_pdb("lig_out.pdb");
 
my @centers = ( 
                V( -53.0534910516002, -30.3815186260294, -29.8956292147067 ),
);
 
foreach my $center ( @centers ) {
 
    my $vina = HackaMol::X::Vina -> new(
        receptor       => $receptor,
        ligand         => $ligand,
        center         => $center,
        size           => V( 20, 20, 20 ),
        cpu            => 4,
        exhaustiveness => 12,
        exe            => '~/perl5/apps/vina',
        scratch        => 'tmp',
    );
 
    my $mol = $vina->dock_mol(3); # fill mol with 3 binding configurations
 
    printf ("Score: %6.1f\n", $mol->get_score($_) ) foreach (0 .. $mol->tmax); 
 
    $mol->print_pdb_ts([0 .. $mol->tmax], $fh); 
 
  }
 
  $_->segid("3nt1") foreach $rmol->all_atoms; #for vmd rendering cartoons.. etc
  $rmol->print_pdb("receptor.pdb"); 1;

Score:   -5.9
Score:   -5.8
Score:   -5.7


1


Warning: overwrite lig_out.pdb at reply input line 8.

overwrite receptor.pdb at reply input line 36.


### Running this script converts the pdbqt files to pdbs.

### After running the script, a lig_out.pdb file and a receptor.pdb file is created and this two can be simultaneously visualized on VMD so as to see the different regions where the receptor was best docked onto the ligand.